##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - finetune with XTuner

This notebook demonstrates how to finetune Gemma with XTuner. [XTuner](https://github.com/InternLM/xtuner) is an efficient, flexible and full-featured toolkit for fine-tuning LLM. XTuner wraps the Hugging Face finetuning functionality and provides a simple interface for finetuning. It's very easy to finetune Gemma with XTuner.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Finetune_with_XTuner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.


### Gemma setup on Hugging Face
XTuner uses Hugging Face under the hood. So you will need to:

* Get access to Gemma on [huggingface.co](huggingface.co) by accepting the Gemma license on the Hugging Face page of the specific model, i.e., [Gemma 2B](https://huggingface.co/google/gemma-2b).
* Generate a [Hugging Face access token](https://huggingface.co/docs/hub/en/security-tokens) and configure it as a Colab secret 'HF_TOKEN'.

In [2]:
import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Install XTuner

In [3]:
!pip install -U 'xtuner'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.

## Finetune Gemma

XTuner has many built-in configurations for finetuning various LLMs. Check out the relevant ones for Gemma. If you are curious how they look like or you want to make adjustments, take a look at the [files](https://github.com/InternLM/xtuner/tree/main/xtuner/configs/gemma).

In [4]:
!xtuner list-cfg | grep gemma

gemma_2b_full_alpaca_e3
gemma_2b_it_full_alpaca_e3
gemma_2b_it_qlora_alpaca_e3
gemma_2b_qlora_alpaca_e3
gemma_7b_full_alpaca_e3
gemma_7b_it_full_alpaca_e3
gemma_7b_it_qlora_alpaca_e3
gemma_7b_qlora_alpaca_e3


For demonstration, this notebook finetunes the instruction tuned Gemma 2B model using [QLoRA](https://arxiv.org/abs/2305.14314) and the [Alpaca dataset](https://huggingface.co/datasets/tatsu-lab/alpaca). You can optionally enable DeepSpeed as well.

In [11]:
!xtuner train gemma_2b_it_qlora_alpaca_e3

06/02 03:40:31 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 204097869
    GPU 0: NVIDIA A100-SXM4-40GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.2, V12.2.140
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.3.0+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=comp

### Convert to Hugging Face

Create a folder for storing converted HF model.

In [12]:
!mkdir -p work_dirs/gemma_2b_it_qlora_alpaca_e3_hf

Conver the LoRA adapter to HF.

In [13]:
!xtuner convert pth_to_hf gemma_2b_it_qlora_alpaca_e3 work_dirs/gemma_2b_it_qlora_alpaca_e3/iter_6500.pth work_dirs/gemma_2b_it_qlora_alpaca_e3_hf

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
quantization_config convert to <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>
`low_cpu_mem_usage` was None, now set to True since model is quantized.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100% 2/2 [00:03<00:00,  1.57s/it]
Load PTH model from work_dirs/gemma_2b_it_qlora_alpaca_e3/iter_6500.pth
Saving adapter to work_dirs/gemma_2b_it_qlora_alpaca_e3_hf
Convert LLM to float16
All done!


### Merge LoRA adapter

Create a folder for storing the merged model.

In [14]:
!mkdir -p work_dirs/gemma_2b_it_qlora_alpaca_e3_merged

Merge the model and the LoRA adapter.

In [15]:
!xtuner convert merge google/gemma-2b-it work_dirs/gemma_2b_it_qlora_alpaca_e3_hf work_dirs/gemma_2b_it_qlora_alpaca_e3_merged --max-shard-size 2GB

2024-06-02 04:42:02.310283: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-02 04:42:02.367089: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 04:42:02.367144: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 04:42:02.369043: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-02 04:42:02.377440: I tensorflow/core/platform/cpu_feature_guar

### Upload model to Hugging Face

Load the model from disk.

In [16]:
from transformers import AutoModel

model = AutoModel.from_pretrained(
    "work_dirs/gemma_2b_it_qlora_alpaca_e3_merged", local_files_only=True
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Push the model to HF Hub.

In [17]:
model.push_to_hub("gemma-2-finetuned-model-xtuner")

model-00001-of-00003.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/windmaple/gemma-2-finetuned-model-xtuner/commit/173d08858a594ed07939f68abb9050f6ceccdf61', commit_message='Upload model', commit_description='', oid='173d08858a594ed07939f68abb9050f6ceccdf61', pr_url=None, pr_revision=None, pr_num=None)

## Conclusion

This notebook demonstrates how to use XTuner to do instruction tuning for the Gemma 2B IT model. If you want to finetune with another dataset, please check out the XTuner documentation on how to [prepare your own dataset](https://github.com/InternLM/xtuner/blob/main/docs/en/user_guides/dataset_prepare.md).